Importing common libraries 

In [1]:
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd

!pip install folium
import folium
import  json  # library to handle JSON files# librar 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


Gather information about Moscow metro stations

In [2]:
# Moscow
wiki = "https://en.wikipedia.org/wiki/List_of_Moscow_Metro_stations"

website_url = requests.get(wiki).text
soup = BeautifulSoup(website_url,'lxml')

my_table = soup.find('table',{'class':'wikitable sortable'})
#print(my_table)

station = []
stationrus = []

for row in my_table.find_all('tr')[1:]:
    station_cell = row.find_all('td')[0] 
    station.append(station_cell.text)

for row in my_table.find_all('tr')[1:]:
    stationrus_cell = row.find_all('td')[1]
    stationrus.append(stationrus_cell.text)


dfm = pd.DataFrame(
{'Station': station,
'Stationrus': stationrus
})

dfm['Station']=dfm['Station'].str.replace('\n','')
dfm['Stationrus']=dfm['Stationrus'].str.replace('\n','')

# Clean data 
dfm = dfm.drop_duplicates(subset=['Station'], keep=False)
dfm['Address'] =['Метро, '] + dfm['Stationrus']
dfm.set_index('Address', inplace=True)
dfm.head(2)

,Station,Stationrus
Address,,
"Метро, Черкизовская",Cherkizovskaya,Черкизовская
"Метро, Преображенская площадь",Preobrazhenskaya Ploshchad,Преображенская площадь


In [3]:
#my Google API key
API_key = 'My API_key'

In [4]:
#Find longitute, latitude and administrativ regions of Moscow metro stations
for pcode in dfm.index:
    url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(API_key, pcode)

    response = requests.get(url).json() # get response
    try:
        geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
        address_data = response['results'][0]['plus_code'] #get administrativ district
        latitude = geographical_data['lat']
        longitude = geographical_data['lng']
        address = address_data['compound_code']
       
        dfm.loc[dfm.index == pcode, 'Latitute']= latitude
        dfm.loc[dfm.index == pcode, 'Longitude']= longitude
        dfm.loc[dfm.index == pcode, 'Okrug']= address
        
        
    except:
        pass
#Take only stations from South administrativ regions
dfm = dfm[dfm['Okrug'].str.contains('South-Eastern', na = False)]
dfm.head(1)

,Station,Stationrus,Latitute,Longitude,Okrug
Address,,,,,
"Метро, Пролетарская",Proletarskaya,Пролетарская,55.731355,37.663879,"PMJ7+GH South-Eastern Administrative District,..."


In [5]:
#dfm = dfm.reset_index(level=0, inplace=True)
dfm

,Station,Stationrus,Latitute,Longitude,Okrug
Address,,,,,
"Метро, Пролетарская",Proletarskaya,Пролетарская,55.731355,37.663879,"PMJ7+GH South-Eastern Administrative District,..."
"Метро, Волгоградский проспект",Volgogradsky Prospekt,Волгоградский проспект,55.725044,37.686638,"PMGP+2M South-Eastern Administrative District,..."
"Метро, Текстильщики",Tekstilshchiki,Текстильщики,55.708972,37.733440,"PP5M+H9 South-Eastern Administrative District,..."
"Метро, Кузьминки",Kuzminki,Кузьминки,55.705137,37.766916,"PQ48+3Q South-Eastern Administrative District,..."
"Метро, Рязанский проспект",Ryazansky Prospekt,Рязанский проспект,55.716654,37.792893,"PQ8V+M5 South-Eastern Administrative District,..."
"Метро, Выхино",Vykhino,Выхино,55.716118,37.817089,"PR88+CR South-Eastern Administrative District,..."
"Метро, Лермонтовский проспект",Lermontovsky Prospekt,Лермонтовский проспект,55.702222,37.851881,"PV22+VQ South-Eastern Administrative District,..."
"Метро, Жулебино",Zhulebino,Жулебино,55.684263,37.854830,"MVM3+PW South-Eastern Administrative District,..."
"Метро, Авиамоторная",Aviamotornaya,Авиамоторная,55.751077,37.717072,"QP28+CR South-Eastern Administrative District,..."


In [6]:
dfm.drop('Okrug', axis=1, inplace=True)

In [7]:
dfm.head()

,Station,Stationrus,Latitute,Longitude
Address,,,,
"Метро, Пролетарская",Proletarskaya,Пролетарская,55.731355,37.663879
"Метро, Волгоградский проспект",Volgogradsky Prospekt,Волгоградский проспект,55.725044,37.686638
"Метро, Текстильщики",Tekstilshchiki,Текстильщики,55.708972,37.733440
"Метро, Кузьминки",Kuzminki,Кузьминки,55.705137,37.766916
"Метро, Рязанский проспект",Ryazansky Prospekt,Рязанский проспект,55.716654,37.792893


In [8]:
wiki1 = "https://en.wikipedia.org/wiki/Districts_of_Yerevan"

website_url = requests.get(wiki1).text
soup = BeautifulSoup(website_url,'lxml')

my_table = soup.find('table',{'class':'wikitable sortable'})
#print(my_table)

statione = []
stationarm = []

for row in my_table.find_all('tr')[1:]:
    statione_cell = row.find_all('td')[0] 
    statione.append(statione_cell.text)

for row in my_table.find_all('tr')[1:]:
    stationarm_cell = row.find_all('td')[1]
    stationarm.append(stationarm_cell.text)


dfe = pd.DataFrame(
{'Station': statione,
'Stationrus': stationarm
})

dfe['Station']=dfe['Station'].str.replace('\n','')
dfe['Stationrus']=dfe['Stationrus'].str.replace('\n','')

dfe['Address'] = dfe['Station'] + [', Yerevan'] 
dfe.set_index('Address', inplace=True)

dfe = dfe.drop(['Ajapnyak, Yerevan', 'Erebuni, Yerevan', 'Nubarashen, Yerevan', 'Shengavit, Yerevan', 'Nork-Marash, Yerevan'])
dfe.head()

,Station,Stationrus
Address,,
"Arabkir, Yerevan",Arabkir,Արաբկիր
"Avan, Yerevan",Avan,Ավան
"Davtashen, Yerevan",Davtashen,Դավթաշեն
"Kanaker-Zeytun, Yerevan",Kanaker-Zeytun,Քանաքեռ-Զեյթուն
"Kentron, Yerevan",Kentron,Կենտրոն


In [9]:
#Find longitute and latitude regions of Yerevan
for pcode in dfe.index:
    url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(API_key, pcode)

    response = requests.get(url).json() # get response
    try:
        geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
        #address_data = response['results'][0]['plus_code'] #get administrativ district
        latitude = geographical_data['lat']
        longitude = geographical_data['lng']
       # address = address_data['compound_code']
       
        dfe.loc[dfe.index == pcode, 'Latitute']= latitude
        dfe.loc[dfe.index == pcode, 'Longitude']= longitude
        #dfe.loc[df.index == pcode, 'Address']= address
        
        
    except:
        pass

In [10]:
dfe

,Station,Stationrus,Latitute,Longitude
Address,,,,
"Arabkir, Yerevan",Arabkir,Արաբկիր,40.210123,44.509193
"Avan, Yerevan",Avan,Ավան,40.218183,44.571770
"Davtashen, Yerevan",Davtashen,Դավթաշեն,40.220299,44.488189
"Kanaker-Zeytun, Yerevan",Kanaker-Zeytun,Քանաքեռ-Զեյթուն,40.205149,44.540573
"Kentron, Yerevan",Kentron,Կենտրոն,40.179246,44.509537
"Malatia-Sebastia, Yerevan",Malatia-Sebastia,Մալաթիա-Սեբաստիա,40.168645,44.457492
"Nor Nork, Yerevan",Nor Nork,Նոր Նորք,40.189923,44.582483


In [11]:
dfe

,Station,Stationrus,Latitute,Longitude
Address,,,,
"Arabkir, Yerevan",Arabkir,Արաբկիր,40.210123,44.509193
"Avan, Yerevan",Avan,Ավան,40.218183,44.571770
"Davtashen, Yerevan",Davtashen,Դավթաշեն,40.220299,44.488189
"Kanaker-Zeytun, Yerevan",Kanaker-Zeytun,Քանաքեռ-Զեյթուն,40.205149,44.540573
"Kentron, Yerevan",Kentron,Կենտրոն,40.179246,44.509537
"Malatia-Sebastia, Yerevan",Malatia-Sebastia,Մալաթիա-Սեբաստիա,40.168645,44.457492
"Nor Nork, Yerevan",Nor Nork,Նոր Նորք,40.189923,44.582483


In [12]:
df = dfe.append(dfm)
df.shape

(25, 4)

In [13]:
df.reset_index(level=0, inplace=True)
df.head()

,Address,Station,Stationrus,Latitute,Longitude
0,"Arabkir, Yerevan",Arabkir,Արաբկիր,40.210123,44.509193
1,"Avan, Yerevan",Avan,Ավան,40.218183,44.571770
2,"Davtashen, Yerevan",Davtashen,Դավթաշեն,40.220299,44.488189
3,"Kanaker-Zeytun, Yerevan",Kanaker-Zeytun,Քանաքեռ-Զեյթուն,40.205149,44.540573
4,"Kentron, Yerevan",Kentron,Կենտրոն,40.179246,44.509537


In [14]:
address = 'Метро, Кузьминки'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 55.7047894, 37.7676828.


In [15]:
map_tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, station in zip(df['Latitute'], df['Longitude'], df['Station']):
    label = '{}'.format(station)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=False).add_to(map_tor)  
    
map_tor

In [16]:
CLIENT_ID = 'MY_CLIENT_ID'
CLIENT_SECRET = 'MY_CLIENT_SECRET'
VERSION = '21082018'

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
LIMIT=100
locs = df['Address']
lats = df['Latitute']
lons = df['Longitude']
venues = getNearbyVenues(locs, lats, lons,400)

Arabkir, Yerevan
Avan, Yerevan
Davtashen, Yerevan
Kanaker-Zeytun, Yerevan
Kentron, Yerevan
Malatia-Sebastia, Yerevan
Nor Nork, Yerevan
Метро, Пролетарская
Метро, Волгоградский проспект
Метро, Текстильщики
Метро, Кузьминки
Метро, Рязанский проспект
Метро, Выхино
Метро, Лермонтовский проспект
Метро, Жулебино
Метро, Авиамоторная
Метро, Кожуховская
Метро, Печатники
Метро, Волжская
Метро, Люблино
Метро, Братиславская
Метро, Марьино
Метро, Нижегородская
Метро, Новохохловская
Метро, Угрешская


In [19]:
venues = venues[venues['Venue Category'] != 'Neighborhood']
moscow_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
moscow_onehot.insert (0,'Neighborhood',venues['Neighborhood'])
moscow_onehot.head()

,Neighborhood,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bath House,...,Theater,Tour Provider,Tourist Information Center,Track,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Wine Shop,Winery,Wings Joint,Yoga Studio
0,"Arabkir, Yerevan",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Arabkir, Yerevan",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Arabkir, Yerevan",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Arabkir, Yerevan",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Avan, Yerevan",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
moscow_onehot.shape

(338, 117)

In [21]:
moscow_grouped = moscow_onehot.groupby('Neighborhood').mean().reset_index()
moscow_grouped

,Neighborhood,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bath House,...,Theater,Tour Provider,Tourist Information Center,Track,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Wine Shop,Winery,Wings Joint,Yoga Studio
0,"Arabkir, Yerevan",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,"Avan, Yerevan",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,"Davtashen, Yerevan",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,"Kanaker-Zeytun, Yerevan",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,"Kentron, Yerevan",0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.038462,0.000000,0.000000
5,"Malatia-Sebastia, Yerevan",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
6,"Nor Nork, Yerevan",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7,"Метро, Авиамоторная",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.032258,0.000000,0.0,0.000000,0.000000,0.032258,0.000000
8,"Метро, Братиславская",0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,...,0.058824,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
9,"Метро, Волгоградский проспект",0.071429,0.000000,0.0,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.071429


In [22]:
num_top_venues = 5

for hood in moscow_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = moscow_grouped[moscow_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arabkir, Yerevan----
                   venue  freq
0             Restaurant  0.25
1      Convenience Store  0.25
2                  Hotel  0.25
3  Performing Arts Venue  0.25
4                 Arcade  0.00


----Avan, Yerevan----
                  venue  freq
0  Gym / Fitness Center  0.33
1            Shoe Store  0.33
2           Supermarket  0.33
3             Multiplex  0.00
4            Playground  0.00


----Davtashen, Yerevan----
                     venue  freq
0              Pizza Place  0.33
1  Comfort Food Restaurant  0.33
2                 Gym Pool  0.33
3                   Arcade  0.00
4          Other Nightlife  0.00


----Kanaker-Zeytun, Yerevan----
               venue  freq
0  Electronics Store   1.0
1             Arcade   0.0
2    Other Nightlife   0.0
3              Plaza   0.0
4         Playground   0.0


----Kentron, Yerevan----
                  venue  freq
0                  Café  0.15
1                 Hotel  0.12
2           Art Gallery  0.08
3  Fast Food Re

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
import numpy as np

In [25]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = moscow_grouped['Neighborhood']

for ind in np.arange(moscow_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(moscow_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(25, 11)

In [26]:
kclusters=5
moscow_grouped_clustering = moscow_grouped.drop(columns='Neighborhood')
kmeans_tot = KMeans(n_clusters=kclusters, random_state=0).fit(moscow_grouped_clustering)
kmeans_tot.cluster_centers_

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.33333333,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [27]:
kmeans_tot.labels_.shape
df.shape

(25, 5)

In [28]:
moscow_merged = df
moscow_merged['Cluster Labels'] = kmeans_tot.labels_
moscow_merged = moscow_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Address')
moscow_merged

,Address,Station,Stationrus,Latitute,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Arabkir, Yerevan",Arabkir,Արաբկիր,40.210123,44.509193,2,Convenience Store,Restaurant,Hotel,Performing Arts Venue,Yoga Studio,Furniture / Home Store,Diner,Dive Spot,Eastern European Restaurant,Electronics Store
1,"Avan, Yerevan",Avan,Ավան,40.218183,44.571770,0,Shoe Store,Gym / Fitness Center,Supermarket,Yoga Studio,Gastropub,Dessert Shop,Diner,Dive Spot,Eastern European Restaurant,Electronics Store
2,"Davtashen, Yerevan",Davtashen,Դավթաշեն,40.220299,44.488189,4,Pizza Place,Comfort Food Restaurant,Gym Pool,Yoga Studio,Gastropub,Diner,Dive Spot,Eastern European Restaurant,Electronics Store,Fast Food Restaurant
3,"Kanaker-Zeytun, Yerevan",Kanaker-Zeytun,Քանաքեռ-Զեյթուն,40.205149,44.540573,1,Electronics Store,Yoga Studio,Gourmet Shop,Dessert Shop,Diner,Dive Spot,Eastern European Restaurant,Fast Food Restaurant,Flower Shop,Food & Drink Shop
4,"Kentron, Yerevan",Kentron,Կենտրոն,40.179246,44.509537,2,Café,Hotel,Fast Food Restaurant,Art Gallery,Restaurant,Coffee Shop,Bookstore,Grocery Store,Mediterranean Restaurant,Plaza
5,"Malatia-Sebastia, Yerevan",Malatia-Sebastia,Մալաթիա-Սեբաստիա,40.168645,44.457492,1,Electronics Store,Yoga Studio,Gourmet Shop,Dessert Shop,Diner,Dive Spot,Eastern European Restaurant,Fast Food Restaurant,Flower Shop,Food & Drink Shop
6,"Nor Nork, Yerevan",Nor Nork,Նոր Նորք,40.189923,44.582483,3,Restaurant,Health & Beauty Service,Yoga Studio,Gastropub,Dessert Shop,Diner,Dive Spot,Eastern European Restaurant,Electronics Store,Fast Food Restaurant
7,"Метро, Пролетарская",Proletarskaya,Пролетарская,55.731355,37.663879,2,Coffee Shop,Miscellaneous Shop,Hobby Shop,Pharmacy,Eastern European Restaurant,Café,Steakhouse,Dessert Shop,Vegetarian / Vegan Restaurant,History Museum
8,"Метро, Волгоградский проспект",Volgogradsky Prospekt,Волгоградский проспект,55.725044,37.686638,2,Photography Studio,Yoga Studio,Shopping Mall,Auto Workshop,Café,Dance Studio,Electronics Store,Gastropub,Gym / Fitness Center,Luggage Store
9,"Метро, Текстильщики",Tekstilshchiki,Текстильщики,55.708972,37.733440,2,Mobile Phone Shop,Sushi Restaurant,Track,Theater,Restaurant,Flower Shop,Movie Theater,Skating Rink,Pool,Eastern European Restaurant


In [29]:
# create map 
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(moscow_merged['Latitute'], moscow_merged['Longitude'], moscow_merged['Address'], moscow_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [30]:
# create map Yerevan
map_clusters = folium.Map(location=[40.177200, 44.503490], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(moscow_merged['Latitute'], moscow_merged['Longitude'], moscow_merged['Address'], moscow_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters